In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def clean_data(df):
    df['IA_abs'] = np.abs(df['IA_50RT'])
    df['pathNorm'] = df['pathlength'] / df['straightlength']
    df['xTargetabs'] = np.abs(df['xTargetEnd'])
    return df

In [35]:
# Save data to CSV with numeric column filtering and handling object columns
def save_data_to_csv(df, results_dir, varlist):
    # Filter out non-numeric columns to prevent mean aggregation errors
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    
    # Group by specified columns and compute mean on numeric columns only
    df_means = df.groupby(['group', "visit", 'studyid', 'subject', 'day', 'Condition', 'Affected'])[numeric_columns].mean().reset_index()
    df_means.to_csv(os.path.join(results_dir, 'means_bysubject.csv'), index=False)

    # Same logic for group by with 'Duration'
    print(df.columns)
    df_meansdur = df.groupby(['group', "visit", 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration'])[numeric_columns].mean().reset_index()
    df_meansdur.to_csv(os.path.join(results_dir, 'means_bysubjectandduration.csv'), index=False)

    # Save Day 1 means
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + varlist].to_csv('Day1_means_bysubject.csv', index=False)

    # Save Day 2 means (if needed)
    df2_means = df_means[df_means['day'] == 'Day2']
    return df_means, df_meansdur


In [10]:

def long_to_wide(df_means, varlist, results_dir):
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_wide = df1_means[['subject', "studyid", 'group', 'day', 'Condition', 'Affected'] + varlist].pivot_table(
        index=["subject", "studyid", "group", "day"],
        columns=['Condition', 'Affected'],
        values=varlist)
    df1_wide.sort_values(['group', 'subject'], ascending=True).to_csv(os.path.join(results_dir, 'Day1_means_bysubject_wide.csv'))
    return df1_wide

In [11]:
def create_excel_sheets(df_means, df_meansdur, results_dir, varlist, total_ids=88, max_days=5):
    all_ids = ['cpvib' + str(item).zfill(3) for item in range(1, total_ids + 1)]
    exceltitle = os.path.join(results_dir, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
    exceltitle2 = os.path.join(results_dir, 'UL_KINARM_Mastersheet_Long_Format.xlsx')
    
    for thisday in range(1, max_days + 1):
        # Wide format per day
        df1_means = df_means[df_means['day'] == 'Day' + str(thisday)]
        df1_wide = df1_means[['subject', "visit", "studyid", 'group', 'day', 'Condition', 'Affected'] + varlist].pivot_table(
            index=["subject", "visit", "studyid", "group", "day"],
            columns=['Condition', 'Affected'],
            values=varlist)
        df1_wide.columns = df1_wide.columns.to_flat_index()
        df1_wide = df1_wide.reset_index(level=["subject", "visit", "group", "day"])
        missing = list(set(all_ids) - set(df1_wide.index.values))
        df1_wide = df1_wide.reindex(df1_wide.index.union(missing))
        df1_wide.index.name = 'NMSKL_ID'
        df1_wide = df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'})
        
        # Wide format per day by duration
        df1_meansdur = df_meansdur[df_meansdur['day'] == 'Day' + str(thisday)]
        df1_widedur = df1_meansdur[['subject', "visit", "studyid", 'group', 'day', 'Condition', 'Affected', 'Duration'] + varlist].pivot_table(
            index=["subject", "visit", "studyid", "group", "day"],
            columns=['Condition', 'Affected', 'Duration'],
            values=varlist)
        df1_widedur.columns = df1_widedur.columns.to_flat_index()
        df1_widedur = df1_widedur.reset_index(level=["subject", "visit", "group", "day"])
        missing = list(set(all_ids) - set(df1_widedur.index.values))
        df1_widedur = df1_widedur.reindex(df1_wide.index.union(missing))
        df1_widedur.index.name = 'NMSKL_ID'
        
        # Combine and save to Excel
        df_combo = pd.concat([df1_wide, df1_widedur], axis=1, join="inner")
        with pd.ExcelWriter(exceltitle, engine="openpyxl", mode='a' if thisday > 1 else 'w') as writer: 
            df_combo.to_excel(writer, sheet_name=f'Day{thisday}_Master_Formatted')
    
    # Save all days' duration data
    with pd.ExcelWriter(exceltitle2) as writer:
        df_meansdur.to_excel(writer, sheet_name='AllDays_Master_Formatted')

# Main execution


In [21]:
def main():
    all_df = pd.read_csv('all_processed_df.csv')
    BASE_DIR = r'C:\Users\LibraryUser\Downloads\Fall2024/BrainAndAction\CP\CP'
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    RESULTS_DIR = os.path.join(BASE_DIR, 'resv2')
    print(f"Results Dir : {RESULTS_DIR}")
    results_dir = RESULTS_DIR

    varlist = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']
    all_df = clean_data(all_df)
    df_means, df_meansdur = save_data_to_csv(all_df, results_dir, varlist)

    # Convert long to wide for Day 1
    df1_wide = long_to_wide(df_means, varlist, results_dir)

    # Create Owais-style Excel sheets
    create_excel_sheets(df_means, df_meansdur, results_dir,varlist)
    print("Success")

if __name__ == "main":
    main()

In [22]:
# main()

In [23]:
all_df = pd.read_csv('all_processed_df.csv')

In [25]:
all_df.columns
all_df = clean_data(all_df)

In [42]:
    all_df = pd.read_csv('all_processed_df.csv')
    BASE_DIR = r'C:\Users\LibraryUser\Downloads\Fall2024/BrainAndAction\CP\CP'
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    RESULTS_DIR = os.path.join(BASE_DIR, 'resv2')
    print(f"Results Dir : {RESULTS_DIR}")
    results_dir = RESULTS_DIR

Results Dir : C:\Users\LibraryUser\Downloads\Fall2024/BrainAndAction\CP\CP\resv2


In [29]:
all_df.size


998400

In [36]:
varlist = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']
all_df = clean_data(all_df)
df_means, df_meansdur = save_data_to_csv(all_df, results_dir, varlist)

Index(['Condition', 'Affected', 'TP', 'Duration', 'Accuracy', 'FeedbackTime',
       'RT', 'CT', 'velPeak', 'xPosError', 'minDist', 'targetDist', 'handDist',
       'straightlength', 'pathlength', 'targetlength', 'cursorX', 'cursorY',
       'IA_RT', 'IA_50RT', 'RTalt', 'IA_RTalt', 'maxpathoffset',
       'meanpathoffset', 'xTargetEnd', 'yTargetEnd', 'EndPointError', 'IDE',
       'PLR', 'PLR_2', 'isCurveAround', 'MT', 'subject', 'age', 'visit', 'day',
       'studyid', 'group', 'pathratio', 'IA_abs', 'pathNorm', 'xTargetabs'],
      dtype='object')


ValueError: cannot insert Duration, already exists

In [37]:
import os
import numpy as np
import pandas as pd

# Calculate additional columns
all_df['IA_abs'] = np.abs(all_df['IA_50RT'])
all_df['pathNorm'] = all_df['pathlength'] / all_df['straightlength']
all_df['xTargetabs'] = np.abs(all_df['xTargetEnd'])

# Variables used for data extraction
varlist = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']

# Save grouped data to CSV files
results_dir = 'results_directory'  # Ensure this directory is defined

df_means = all_df.groupby(['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected']).mean().reset_index()
df_means.to_csv(os.path.join(results_dir, 'means_bysubject.csv'), index=False)

df_meansdur = all_df.groupby(['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration']).mean().reset_index()
df_meansdur.to_csv(os.path.join(results_dir, 'means_bysubjectandduration.csv'), index=False)

# Filter and save Day 1 and Day 2 means
hit_df = all_df[all_df['Accuracy'] == 1]

df1_means = df_means[df_means['day'] == 'Day1']
df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + varlist].to_csv('Day1_means_bysubject.csv', index=False)

df2_means = df_means[df_means['day'] == 'Day2']

# Convert long data to wide format for Day 1
df1_wide = df1_means.pivot_table(
    index=['subject', 'studyid', 'group', 'day'],
    columns=['Condition', 'Affected'],
    values=varlist
).reset_index()

df1_wide.sort_values(['group', 'subject'], ascending=True).to_csv(os.path.join(results_dir, 'Day1_means_bysubject_wide.csv'), index=False)

# Excel Sheet Creation for Multiple Days
total_ids = 88
all_ids = ['cpvib' + str(item).zfill(3) for item in range(1, total_ids + 1)]

exceltitle = os.path.join(results_dir, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
exceltitle2 = os.path.join(results_dir, 'UL_KINARM_Mastersheet_Long_Format.xlsx')

max_days = 5
for day_num in range(1, max_days + 1):
    current_day = f'Day{day_num}'
    
    # Process Day-Wise Wide Format Data
    df_day_means = df_means[df_means['day'] == current_day]
    df_day_wide = df_day_means.pivot_table(
        index=['subject', 'visit', 'studyid', 'group', 'day'],
        columns=['Condition', 'Affected'],
        values=varlist
    ).reset_index()
    df_day_wide.columns = df_day_wide.columns.to_flat_index()
    df_day_wide = df_day_wide.set_index('subject').reindex(all_ids).reset_index()
    
    df_day_wide.index.name = 'NMSKL_ID'
    df_day_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

    # Process Day-Wise Data by Duration
    df_day_meansdur = df_meansdur[df_meansdur['day'] == current_day]
    df_day_widedur = df_day_meansdur.pivot_table(
        index=['subject', 'visit', 'studyid', 'group', 'day'],
        columns=['Condition', 'Affected', 'Duration'],
        values=varlist
    ).reset_index()
    df_day_widedur.columns = df_day_widedur.columns.to_flat_index()
    df_day_widedur = df_day_widedur.set_index('subject').reindex(all_ids).reset_index()
    
    df_day_widedur.index.name = 'NMSKL_ID'
    df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

    # Combine the data and write to Excel
    df_combo = pd.concat([df_day_wide, df_day_widedur], axis=1, join="inner")

    if day_num == 1:
        with pd.ExcelWriter(exceltitle) as writer:
            df_combo.to_excel(writer, sheet_name=f'{current_day}_Master_Formatted', index=False)
    else:
        with pd.ExcelWriter(exceltitle, engine="openpyxl", mode='a') as writer:
            df_combo.to_excel(writer, sheet_name=f'{current_day}_Master_Formatted', index=False)

# Save long format Excel file for all days
df_meansdur.to_excel(exceltitle2, sheet_name='AllDays_Master_Formatted', index=False)

# Potential Fixes and Improvements:
# - Defined a results directory for output files to avoid errors.
# - Added explicit reset_index() calls to ensure proper reshaping.
# - Replaced ambiguous .loc[] indexing with more readable conditional indexing.
# - Improved file naming for consistency and removed unnecessary comments.
# - Ensured to specify index=False to avoid writing index columns to CSV/Excel.
# - Added handling for missing IDs in wide dataframes to ensure consistent shape.

TypeError: agg function failed [how->mean,dtype->object]

In [45]:
import os
import numpy as np
import pandas as pd

# Define constants
RESULTS_DIR = r'C:\Users\LibraryUser\Downloads\Fall2024/BrainAndAction\CP\CP\resv2'  # Ensure this directory is defined
TOTAL_IDS = 88
ALL_IDS = ['cpvib' + str(item).zfill(3) for item in range(1, TOTAL_IDS + 1)]
MAX_DAYS = 5
VARLIST = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']

# Helper functions
def calculate_additional_columns(df):
    df['IA_abs'] = np.abs(df['IA_50RT'])
    df['pathNorm'] = df['pathlength'] / df['straightlength']
    df['xTargetabs'] = np.abs(df['xTargetEnd'])
    return df

def save_grouped_data(df, group_cols, filename):
    grouped_df = df.groupby(group_cols).mean(numeric_only=True).reset_index()
    grouped_df.to_csv(os.path.join(RESULTS_DIR, filename), index=False)
    return grouped_df

def pivot_data_to_wide(df, index_cols, pivot_cols, values_cols):
    wide_df = df.pivot_table(index=index_cols, columns=pivot_cols, values=values_cols).reset_index()
    wide_df.columns = wide_df.columns.to_flat_index()
    return wide_df

def reindex_with_missing_ids(df, all_ids):
    df = df.set_index('subject').reindex(all_ids).reset_index()
    df.index.name = 'NMSKL_ID'
    return df

def save_excel(df, filepath, sheet_name, mode='w'):
    with pd.ExcelWriter(filepath, engine='openpyxl', mode=mode) as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Main processing
def main():
    # Load and calculate additional columns
    all_df = pd.read_csv('all_processed_df.csv')  # Assuming input data is loaded from a CSV
    all_df = calculate_additional_columns(all_df)

    # Save grouped data to CSV files
    df_means = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected'], 'means_bysubject.csv')
    df_meansdur = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration'], 'means_bysubjectandduration.csv')

    # Filter and save Day 1 means
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + VARLIST].to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject.csv'), index=False)

    # Convert long data to wide format for Day 1 and save
    df1_wide = pivot_data_to_wide(df1_means, ['subject', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df1_wide = reindex_with_missing_ids(df1_wide, ALL_IDS)
    df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)
    df1_wide.sort_values(['group', 'KINARM_ID'], ascending=True).to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject_wide.csv'), index=False)

    # Excel Sheet Creation for Multiple Days
    exceltitle = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
    exceltitle2 = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Long_Format.xlsx')

    for day_num in range(1, MAX_DAYS + 1):
        current_day = f'Day{day_num}'

        # Process Day-Wise Wide Format Data
        df_day_means = df_means[df_means['day'] == current_day]
        df_day_wide = pivot_data_to_wide(df_day_means, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
        df_day_wide = reindex_with_missing_ids(df_day_wide, ALL_IDS)
        df_day_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

        # Process Day-Wise Data by Duration
        df_day_meansdur = df_meansdur[df_meansdur['day'] == current_day]
        df_day_widedur = pivot_data_to_wide(df_day_meansdur, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected', 'Duration'], VARLIST)
        df_day_widedur = reindex_with_missing_ids(df_day_widedur, ALL_IDS)
        df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

        # Combine the data and write to Excel
        df_combo = pd.concat([df_day_wide, df_day_widedur], axis=1, join="inner")

        if day_num == 1:
            save_excel(df_combo, exceltitle, f'{current_day}_Master_Formatted', mode='w')
        else:
            save_excel(df_combo, exceltitle, f'{current_day}_Master_Formatted', mode='a')

    # Save long format Excel file for all days
    save_excel(df_meansdur, exceltitle2, 'AllDays_Master_Formatted', mode='w')

if __name__ == "__main__":
    main()

KeyError: "None of ['subject'] are in the columns"

In [46]:
RESULTS_DIR

'C:\\Users\\LibraryUser\\Downloads\\Fall2024/BrainAndAction\\CP\\CP\\resv2'

In [48]:
import os
import numpy as np
import pandas as pd

# Define constants
# RESULTS_DIR = 'results_directory'  # Ensure this directory is defined
TOTAL_IDS = 88
ALL_IDS = ['cpvib' + str(item).zfill(3) for item in range(1, TOTAL_IDS + 1)]
MAX_DAYS = 5
VARLIST = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']

# Helper functions
def calculate_additional_columns(df):
    df['IA_abs'] = np.abs(df['IA_50RT'])
    df['pathNorm'] = df['pathlength'] / df['straightlength']
    df['xTargetabs'] = np.abs(df['xTargetEnd'])
    return df

def save_grouped_data(df, group_cols, filename):
    grouped_df = df.groupby(group_cols).mean(numeric_only=True).reset_index()
    grouped_df.to_csv(os.path.join(RESULTS_DIR, filename), index=False)
    return grouped_df

def pivot_data_to_wide(df, index_cols, pivot_cols, values_cols):
    wide_df = df.pivot_table(index=index_cols, columns=pivot_cols, values=values_cols).reset_index()
    wide_df.columns = ['_'.join(filter(None, map(str, col))).strip() for col in wide_df.columns.values]
    return wide_df

def reindex_with_missing_ids(df, all_ids):
    df['subject'] = df['subject'].astype(str)
    df = df.set_index('subject').reindex(all_ids).reset_index()
    df.index.name = 'NMSKL_ID'
    return df

def save_excel(df, filepath, sheet_name, mode='w'):
    with pd.ExcelWriter(filepath, engine='openpyxl', mode=mode) as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Main processing
def main():
    # Load and calculate additional columns
    all_df = pd.read_csv('all_processed_df.csv')  # Assuming input data is loaded from a CSV
    all_df = calculate_additional_columns(all_df)

    # Save grouped data to CSV files
    df_means = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected'], 'means_bysubject.csv')
    df_meansdur = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration'], 'means_bysubjectandduration.csv')

    # Filter and save Day 1 means
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + VARLIST].to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject.csv'), index=False)

    # Convert long data to wide format for Day 1 and save
    df1_wide = pivot_data_to_wide(df1_means, ['subject', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df1_wide = reindex_with_missing_ids(df1_wide, ALL_IDS)
    df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)
    df1_wide.sort_values(['group', 'KINARM_ID'], ascending=True).to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject_wide.csv'), index=False)

    # Excel Sheet Creation for Multiple Days
    exceltitle = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
    exceltitle2 = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Long_Format.xlsx')

    for day_num in range(1, MAX_DAYS + 1):
        current_day = f'Day{day_num}'

        # Process Day-Wise Wide Format Data
        df_day_means = df_means[df_means['day'] == current_day]
        df_day_wide = pivot_data_to_wide(df_day_means, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
        df_day_wide = reindex_with_missing_ids(df_day_wide, ALL_IDS)
        df_day_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

        # Process Day-Wise Data by Duration
        df_day_meansdur = df_meansdur[df_meansdur['day'] == current_day]
        df_day_widedur = pivot_data_to_wide(df_day_meansdur, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected', 'Duration'], VARLIST)
        df_day_widedur = reindex_with_missing_ids(df_day_widedur, ALL_IDS)
        df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

        # Combine the data and write to Excel
        df_combo = pd.concat([df_day_wide, df_day_widedur], axis=1, join="outer")

        if day_num == 1:
            save_excel(df_combo, exceltitle, f'{current_day}_Master_Formatted', mode='w')
        else:
            save_excel(df_combo, exceltitle, f'{current_day}_Master_Formatted', mode='a')

    # Save long format Excel file for all days
    save_excel(df_meansdur, exceltitle2, 'AllDays_Master_Formatted', mode='w')

if __name__ == "__main__":
    main()

In [54]:
import os
import numpy as np
import pandas as pd

# Define constant
RESULTS_DIR = r'C:\\Users\\LibraryUser\\Downloads\\Fall2024/BrainAndAction\\CP\\CP\\resv2'

TOTAL_IDS = 88
ALL_IDS = ['cpvib' + str(item).zfill(3) for item in range(1, TOTAL_IDS + 1)]
MAX_DAYS = 5
VARLIST = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']

# Helper functions
def calculate_additional_columns(df):
    df['IA_abs'] = np.abs(df['IA_50RT'])
    df['pathNorm'] = df['pathlength'] / df['straightlength']
    df['xTargetabs'] = np.abs(df['xTargetEnd'])
    return df

def save_grouped_data(df, group_cols, filename):
    grouped_df = df.groupby(group_cols).mean(numeric_only=True).reset_index()
    grouped_df.to_csv(os.path.join(RESULTS_DIR, filename), index=False)
    return grouped_df

def pivot_data_to_wide(df, index_cols, pivot_cols, values_cols):
    wide_df = df.pivot_table(index=index_cols, columns=pivot_cols, values=values_cols)
    wide_df.columns = pd.MultiIndex.from_tuples(wide_df.columns)
    wide_df = wide_df.reset_index()
    return wide_df

def reindex_with_missing_ids(df, all_ids, index_name='NMSKL_ID'):
    df = df.set_index('subject').reindex(all_ids).reset_index()
    df.index.name = index_name
    return df

def save_excel(df, filepath, sheet_name, mode='w'):
    with pd.ExcelWriter(filepath, engine='openpyxl', mode=mode) as writer:
        df.to_excel(writer, sheet_name=sheet_name)

# Main processing
def main():
    # Load and calculate additional columns
    all_df = pd.read_csv('all_processed_df.csv')  # Assuming input data is loaded from a CSV
    all_df = calculate_additional_columns(all_df)

    # Save grouped data to CSV files
    df_means = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected'], 'means_bysubject.csv')
    df_meansdur = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration'], 'means_bysubjectandduration.csv')

    # Filter and save Day 1 means
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + VARLIST].to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject.csv'), index=False)

    # Convert long data to wide format for Day 1 and save
    df1_wide = pivot_data_to_wide(df1_means, ['subject', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df1_wide = reindex_with_missing_ids(df1_wide, ALL_IDS)
    df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)
    df1_wide.sort_values(['group', 'KINARM_ID'], ascending=True).to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject_wide.csv'), index=False)

    # Excel Sheet Creation for Multiple Days
    exceltitle = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
    exceltitle2 = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Long_Format.xlsx')

    for day_num in range(1, MAX_DAYS + 1):
        current_day = f'Day{day_num}'

        # Process Day-Wise Wide Format Data
        df_day_means = df_means[df_means['day'] == current_day]
        df_day_wide = pivot_data_to_wide(df_day_means, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
        df_day_wide = reindex_with_missing_ids(df_day_wide, ALL_IDS)
        df_day_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

        # Process Day-Wise Data by Duration
        df_day_meansdur = df_meansdur[df_meansdur['day'] == current_day]
        df_day_widedur = pivot_data_to_wide(df_day_meansdur, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected', 'Duration'], VARLIST)
        df_day_widedur = reindex_with_missing_ids(df_day_widedur, ALL_IDS)
        df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

        # Combine the data and write to Excel
        df_combo = pd.concat([df_day_wide, df_day_widedur], axis=1, join="outer")

        if day_num == 1:
            save_excel(df_combo, exceltitle, f'{current_day}_Master_Formatted', mode='w')
        else:
            save_excel(df_combo, exceltitle, f'{current_day}_Master_Formatted', mode='a')

    # Save long format Excel file for all days
    save_excel(df_meansdur, exceltitle2, 'AllDays_Master_Formatted', mode='w')

if __name__ == "__main__":
    main()

C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\2852807684.py:77: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)
C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\2852807684.py:77: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)
C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\2852807684.py:77: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)
C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\2852807684.py:77: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.d

In [55]:
import os
import numpy as np
import pandas as pd

# Define constants
RESULTS_DIR = r'C:\\Users\\LibraryUser\\Downloads\\Fall2024/BrainAndAction\\CP\\CP\\resv2'
TOTAL_IDS = 88
ALL_IDS = ['cpvib' + str(item).zfill(3) for item in range(1, TOTAL_IDS + 1)]
MAX_DAYS = 5
VARLIST = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']
DURATION_VALUES = [500, 625, 750, 900]

# Helper functions
def calculate_additional_columns(df):
    df['IA_abs'] = np.abs(df['IA_50RT'])
    df['pathNorm'] = df['pathlength'] / df['straightlength']
    df['xTargetabs'] = np.abs(df['xTargetEnd'])
    return df

def save_grouped_data(df, group_cols, filename):
    grouped_df = df.groupby(group_cols).mean(numeric_only=True).reset_index()
    grouped_df.to_csv(os.path.join(RESULTS_DIR, filename), index=False)
    return grouped_df

def pivot_data_to_wide(df, index_cols, pivot_cols, values_cols):
    wide_df = df.pivot_table(index=index_cols, columns=pivot_cols, values=values_cols)
    wide_df.columns = pd.MultiIndex.from_tuples(wide_df.columns)
    wide_df = wide_df.reset_index()
    return wide_df

def reindex_with_missing_ids(df, all_ids, index_name='NMSKL_ID'):
    df = df.set_index('subject').reindex(all_ids).reset_index()
    df.index.name = index_name
    return df

def save_excel(df, filepath, sheet_name, mode='w'):
    with pd.ExcelWriter(filepath, engine='openpyxl', mode=mode) as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Main processing
def main():
    # Load and calculate additional columns
    all_df = pd.read_csv('all_processed_df.csv')  # Assuming input data is loaded from a CSV
    all_df = calculate_additional_columns(all_df)

    # Save grouped data to CSV files
    df_means = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected'], 'means_bysubject.csv')
    df_meansdur = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration'], 'means_bysubjectandduration.csv')

    # Filter and save Day 1 means
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + VARLIST].to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject.csv'), index=False)

    # Convert long data to wide format for Day 1 and save
    df1_wide = pivot_data_to_wide(df1_means, ['subject', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df1_wide = reindex_with_missing_ids(df1_wide, ALL_IDS)
    df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)
    df1_wide.sort_values(['group', 'KINARM_ID'], ascending=True).to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject_wide.csv'), index=False)

    # Excel Sheet Creation for Multiple Days
    exceltitle = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
    exceltitle2 = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Long_Format.xlsx')

    for day_num in range(1, MAX_DAYS + 1):
        current_day = f'Day{day_num}'

        # Process Day-Wise Wide Format Data
        df_day_means = df_means[df_means['day'] == current_day]
        df_day_wide = pivot_data_to_wide(df_day_means, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
        df_day_wide = reindex_with_missing_ids(df_day_wide, ALL_IDS)
        df_day_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

        # Process Day-Wise Data by Duration
        df_day_meansdur = df_meansdur[df_meansdur['day'] == current_day]
        df_day_widedur = pivot_data_to_wide(df_day_meansdur, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected', 'Duration'], VARLIST)
        df_day_widedur = reindex_with_missing_ids(df_day_widedur, ALL_IDS)
        df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

        # Ensure the correct ordering of columns to match the expected output
        ordered_columns = pd.MultiIndex.from_product([
            ['Accuracy', 'MT', 'RT', 'pathlength', 'velPeak'],
            ['Interception', 'Reaching'],
            ['Less Affected', 'More Affected'],
            DURATION_VALUES + [None]
        ]).dropna().tolist()
        df_day_combo = pd.concat([df_day_wide, df_day_widedur], axis=1, join="outer")
        df_day_combo = df_day_combo.reindex(columns=ordered_columns, fill_value=np.nan)

        if day_num == 1:
            save_excel(df_day_combo, exceltitle, f'{current_day}_Master_Formatted', mode='w')
        else:
            save_excel(df_day_combo, exceltitle, f'{current_day}_Master_Formatted', mode='a')

    # Save long format Excel file for all days
    save_excel(df_meansdur, exceltitle2, 'AllDays_Master_Formatted', mode='w')

if __name__ == "__main__":
    main()

C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\1653975767.py:77: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)


ValueError: cannot handle a non-unique multi-index!

In [58]:
import os
import numpy as np
import pandas as pd

# Define constants
RESULTS_DIR = r'C:\\Users\\LibraryUser\\Downloads\\Fall2024/BrainAndAction\\CP\\CP\\resv2'
TOTAL_IDS = 88
ALL_IDS = ['cpvib' + str(item).zfill(3) for item in range(1, TOTAL_IDS + 1)]
MAX_DAYS = 5
VARLIST = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']
DURATION_VALUES = [500, 625, 750, 900]

# Helper functions
def calculate_additional_columns(df):
    df['IA_abs'] = np.abs(df['IA_50RT'])
    df['pathNorm'] = df['pathlength'] / df['straightlength']
    df['xTargetabs'] = np.abs(df['xTargetEnd'])
    return df

def save_grouped_data(df, group_cols, filename):
    grouped_df = df.groupby(group_cols).mean(numeric_only=True).reset_index()
    grouped_df.to_csv(os.path.join(RESULTS_DIR, filename), index=False)
    return grouped_df

def pivot_data_to_wide(df, index_cols, pivot_cols, values_cols):
    wide_df = df.pivot_table(index=index_cols, columns=pivot_cols, values=values_cols)
    wide_df.columns = pd.MultiIndex.from_tuples(wide_df.columns)
    wide_df = wide_df.reset_index()
    return wide_df

def reindex_with_missing_ids(df, all_ids, index_name='NMSKL_ID'):
    df = df.set_index('subject').reindex(all_ids).reset_index()
    df.index.name = index_name
    return df

def flatten_multiindex_columns(df):
    df.columns = ['_'.join(filter(None, map(str, col))).strip() if isinstance(col, tuple) else col for col in df.columns]
    return df

def save_excel(df, filepath, sheet_name, mode='w'):
    with pd.ExcelWriter(filepath, engine='openpyxl', mode=mode) as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Main processing
def main():
    # Load and calculate additional columns
    all_df = pd.read_csv('all_processed_df.csv')  # Assuming input data is loaded from a CSV
    all_df = calculate_additional_columns(all_df)

    # Save grouped data to CSV files
    df_means = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected'], 'means_bysubject.csv')
    df_meansdur = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration'], 'means_bysubjectandduration.csv')

    # Filter and save Day 1 means
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + VARLIST].to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject.csv'), index=False)

    # Convert long data to wide format for Day 1 and save
    df1_wide = pivot_data_to_wide(df1_means, ['subject', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df1_wide = reindex_with_missing_ids(df1_wide, ALL_IDS)
    df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)
    df1_wide.sort_values(['group', 'KINARM_ID'], ascending=True).to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject_wide.csv'), index=False)

    # Excel Sheet Creation for Multiple Days
    exceltitle = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
    exceltitle2 = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Long_Format.xlsx')

    for day_num in range(1, MAX_DAYS + 1):
        current_day = f'Day{day_num}'

        # Process Day-Wise Wide Format Data
        df_day_means = df_means[df_means['day'] == current_day]
        df_day_wide = pivot_data_to_wide(df_day_means, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
        df_day_wide = reindex_with_missing_ids(df_day_wide, ALL_IDS)
        df_day_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

        # Process Day-Wise Data by Duration
        df_day_meansdur = df_meansdur[df_meansdur['day'] == current_day]
        df_day_widedur = pivot_data_to_wide(df_day_meansdur, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected', 'Duration'], VARLIST)
        df_day_widedur = reindex_with_missing_ids(df_day_widedur, ALL_IDS)
        df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

        # Combine the data and write to Excel
        df_day_combo = pd.concat([df_day_wide, df_day_widedur], axis=1)
        df_day_combo = df_day_combo.loc[:, ~df_day_combo.columns.duplicated()]  # Remove duplicate columns
        df_day_combo = flatten_multiindex_columns(df_day_combo)  # Flatten multi-index columns

        if day_num == 1:
            save_excel(df_day_combo, exceltitle, f'{current_day}_Master_Formatted', mode='w')
        else:
            save_excel(df_day_combo, exceltitle, f'{current_day}_Master_Formatted', mode='a')

    # Save long format Excel file for all days
    df_meansdur = flatten_multiindex_columns(df_meansdur)  # Flatten multi-index columns
    save_excel(df_meansdur, exceltitle2, 'AllDays_Master_Formatted', mode='w')

if __name__ == "__main__":
    main()

C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\3652772879.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)
C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\3652772879.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)
C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\3652772879.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)
C:\Users\LibraryUser\AppData\Local\Temp\ipykernel_11088\3652772879.py:81: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_day_widedur.d

In [60]:
import os
import numpy as np
import pandas as pd

# Define constants
RESULTS_DIR = r'C:\Users\LibraryUser\Downloads\Fall2024\BrainAndAction\CP\CP\resv2'  # Ensure this directory is defined
TOTAL_IDS = 88
ALL_IDS = ['cpvib' + str(item).zfill(3) for item in range(1, TOTAL_IDS + 1)]
MAX_DAYS = 5
VARLIST = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError', 'RTalt', 'IA_abs']

# Helper functions
def calculate_additional_columns(df):
    df['IA_abs'] = np.abs(df['IA_50RT'])
    df['pathNorm'] = df['pathlength'] / df['straightlength']
    df['xTargetabs'] = np.abs(df['xTargetEnd'])
    return df

def save_grouped_data(df, group_cols, filename):
    # Using numeric_only=True to avoid issues with non-numeric columns
    grouped_df = df.groupby(group_cols).mean(numeric_only=True).reset_index()
    grouped_df.to_csv(os.path.join(RESULTS_DIR, filename), index=False)
    return grouped_df

def pivot_data_to_wide(df, index_cols, pivot_cols, values_cols):
    wide_df = df.pivot_table(index=index_cols, columns=pivot_cols, values=values_cols)
    wide_df.columns = wide_df.columns.to_flat_index()
    wide_df = wide_df.reset_index()
    return wide_df

def reindex_with_missing_ids(df, all_ids, index_name='NMSKL_ID'):
    df = df.set_index('subject').reindex(all_ids).reset_index()
    df.index.name = index_name
    return df

def save_excel(df, filepath, sheet_name, mode='w'):
    with pd.ExcelWriter(filepath, engine='openpyxl', mode=mode) as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

def process_and_save_day_data(df_means, df_meansdur, day_num, exceltitle, all_ids):
    current_day = f'Day{day_num}'

    # Process Day-Wise Wide Format Data
    df_day_means = df_means[df_means['day'] == current_day]
    df_day_wide = pivot_data_to_wide(df_day_means, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df_day_wide = reindex_with_missing_ids(df_day_wide, all_ids)
    df_day_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

    # Process Day-Wise Data by Duration
    df_day_meansdur = df_meansdur[df_meansdur['day'] == current_day]
    df_day_widedur = pivot_data_to_wide(df_day_meansdur, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected', 'Duration'], VARLIST)
    df_day_widedur = reindex_with_missing_ids(df_day_widedur, all_ids)
    df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

    # Combine the data and write to Excel
    df_day_combo = pd.concat([df_day_wide, df_day_widedur], axis=1, join="inner")
    if day_num == 1:
        save_excel(df_day_combo, exceltitle, f'{current_day}_Master_Formatted', mode='w')
    else:
        save_excel(df_day_combo, exceltitle, f'{current_day}_Master_Formatted', mode='a')

# Main processing
def main():
    # Load and calculate additional columns
    all_df = pd.read_csv('all_processed_df.csv')  # Assuming input data is loaded from a CSV
    all_df = calculate_additional_columns(all_df)

    # Save grouped data to CSV files
    df_means = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected'], 'means_bysubject.csv')
    df_meansdur = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration'], 'means_bysubjectandduration.csv')

    # Filter and save Day 1 means
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + VARLIST].to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject.csv'), index=False)

    # Convert long data to wide format for Day 1 and save
    df1_wide = pivot_data_to_wide(df1_means, ['subject', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df1_wide = reindex_with_missing_ids(df1_wide, ALL_IDS)
    df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)
    df1_wide.sort_values(['group', 'KINARM_ID'], ascending=True).to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject_wide.csv'), index=False)

    # Excel Sheet Creation for Multiple Days
    exceltitle = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
    exceltitle2 = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Long_Format.xlsx')

    for day_num in range(1, MAX_DAYS + 1):
        process_and_save_day_data(df_means, df_meansdur, day_num, exceltitle, ALL_IDS)

    # Save long format Excel file for all days
    save_excel(df_meansdur, exceltitle2, 'AllDays_Master_Formatted', mode='w')

if __name__ == "__main__":
    main()

In [84]:
import os
import numpy as np
import pandas as pd

# Define constants
RESULTS_DIR = r'C:\Users\LibraryUser\Downloads\Fall2024\BrainAndAction\CP\CP\resv2'  # Ensure this directory is defined
TOTAL_IDS = 88
ALL_IDS = ['cpvib' + str(item).zfill(3) for item in range(1, TOTAL_IDS + 1)]
MAX_DAYS = 5
VARLIST = ['Accuracy','MT','RT','pathlength','velPeak']
# Helper functions
def calculate_additional_columns(df):
    df['pathNorm'] = df['pathlength'] / df['straightlength']
    df['xTargetabs'] = np.abs(df['xTargetEnd'])
    return df

def save_grouped_data(df, group_cols, filename):
    # Using numeric_only=True to avoid issues with non-numeric columns
    grouped_df = df.groupby(group_cols).mean(numeric_only=True).reset_index()
    grouped_df.to_csv(os.path.join(RESULTS_DIR, filename), index=False)
    return grouped_df

def pivot_data_to_wide(df, index_cols, pivot_cols, values_cols):
    wide_df = df.pivot_table(index=index_cols, columns=pivot_cols, values=values_cols)
    wide_df.columns = wide_df.columns.to_flat_index()
    wide_df = wide_df.reset_index()
    return wide_df

def reindex_with_missing_ids(df, all_ids, index_name='NMSKL_ID'):
    df = df.set_index('subject').reindex(all_ids).reset_index()
    df.index.name = index_name
    return df

def save_excel(df, filepath, sheet_name, mode='w'):
    with pd.ExcelWriter(filepath, engine='openpyxl', mode=mode) as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

def process_and_save_day_data(df_means, df_meansdur, day_num, exceltitle, all_ids):
    current_day = f'Day{day_num}'
    varlist = ['Accuracy','MT','RT','pathlength','velPeak']
    # Process Day-Wise Wide Format Data
    df_day_means = df_means[df_means['day'] == current_day]
    df_day_wide = pivot_data_to_wide(df_day_means, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df_day_wide = reindex_with_missing_ids(df_day_wide, all_ids)
    df_day_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

    # Process Day-Wise Data by Duration
    df_day_meansdur = df_meansdur[df_meansdur['day'] == current_day]
    df_day_widedur = pivot_data_to_wide(df_day_meansdur, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected', 'Duration'], VARLIST)
    df_day_widedur = reindex_with_missing_ids(df_day_widedur, all_ids)
    df_day_widedur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

    # Combine the data and write to Excel
    df_day_combo = pd.concat([df_day_wide, df_day_widedur], axis=1, join="inner")
    if day_num == 1:
        save_excel(df_day_combo, exceltitle, f'{current_day}_Master_Formatted', mode='w')
    else:
        save_excel(df_day_combo, exceltitle, f'{current_day}_Master_Formatted', mode='a')

# Main processing
def main():
    # Load and calculate additional columns
    all_df = pd.read_csv('all_processed_df.csv')  # Assuming input data is loaded from a CSV
    all_df = calculate_additional_columns(all_df)

    # Save grouped data to CSV files
    df_means = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected'], 'means_bysubject.csv')
    df_meansdur = save_grouped_data(all_df, ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected', 'Duration'], 'means_bysubjectandduration.csv')

    # Filter and save Day 1 means
    df1_means = df_means[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + VARLIST].to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject.csv'), index=False)

    # Convert long data to wide format for Day 1 and save
    df1_wide = pivot_data_to_wide(df1_means, ['subject', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)
    df1_wide = reindex_with_missing_ids(df1_wide, ALL_IDS)
    df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)
    df1_wide.sort_values(['group', 'KINARM_ID'], ascending=True).to_csv(os.path.join(RESULTS_DIR, 'Day1_means_bysubject_wide.csv'), index=False)

    # Excel Sheet Creation for Multiple Days
    exceltitle = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Auto_Format.xlsx')
    exceltitle2 = os.path.join(RESULTS_DIR, 'UL_KINARM_Mastersheet_Long_Format.xlsx')
    varlist = ['Accuracy','MT','RT','pathlength','velPeak']
    for day_num in range(1, MAX_DAYS + 1):
        process_and_save_day_data(df_means, df_meansdur, day_num, exceltitle, ALL_IDS)

    # Save long format Excel file for all days
    save_excel(df_meansdur, exceltitle2, 'AllDays_Master_Formatted', mode='w')

if __name__ == "__main__":
    main()

In [66]:
all_df = pd.read_csv('all_processed_df.csv')
varlist = ['age', 'Accuracy', 'RT', 'MT', 'velPeak', 'pathlength', 'CT', 'xPosError',
                   # 'targetDist','targetlength'
                   'RTalt', 'IA_abs', 'EndPointError', 'IDE', 'PLR']

In [69]:
all_df = clean_data(all_df)


In [75]:
df_means, df_meansdur, df_stds, df_stdsdur, df1_means, df1_stds, df2_means, df2_stds = save_data_to_csv(
            all_df, varlist)
prepare_excel_export(df_means, df_meansdur,df_stds,df_stdsdur ,RESULTS_DIR)

In [86]:
import os
import pandas as pd
import numpy as np

RESULTS_DIR = r'C:\Users\LibraryUser\Downloads\Fall2024\BrainAndAction\CP\CP\resv2'

def save_data_to_csv(all_df, varlist):
    results_dir = RESULTS_DIR
    group_cols = ['group', 'visit', 'studyid', 'subject', 'day', 'Condition', 'Affected']

    # Select numeric columns
    numeric_cols = all_df.select_dtypes(include=[np.number]).columns
    all_df_numeric = all_df[group_cols + list(numeric_cols)].copy()

    # Group by the relevant columns and calculate the mean
    df_means = all_df_numeric.groupby(group_cols).mean().reset_index()
    df_means.to_csv(os.path.join(results_dir, 'means_bysubject.csv'))

    # Group by the relevant columns including 'Duration' and calculate the mean
    df_meansdur = all_df_numeric.groupby(group_cols + ['Duration']).mean().reset_index()
    df_meansdur.to_csv(os.path.join(results_dir, 'means_bysubjectandduration.csv'))

    # Filter for Accuracy == 1
    hit_df = all_df_numeric.loc[all_df['Accuracy'] == 1]

    # Separate by Day 1 and Day 2 and save as CSV
    df1_means = df_means.loc[df_means['day'] == 'Day1']
    df1_means[['subject', 'group', 'day', 'Condition', 'Affected'] + varlist].to_csv('Day1_means_bysubject.csv')

    df2_means = df_means.loc[df_means['day'] == 'Day2']
    df2_means[['subject', 'group', 'day', 'Condition', 'Affected'] + varlist].to_csv('Day2_means_bysubject.csv')

    # Long to wide format conversion
    df1_wide = df1_means.pivot_table(index=["subject", "studyid", "group", "day"],
                                     columns=['Condition', 'Affected'], values=varlist)
    df2_wide = df2_means.pivot_table(index=["subject", "studyid", "group", "day"],
                                     columns=['Condition', 'Affected'], values=varlist)

    # Sort and save wide format
    df1_wide.sort_values(['group', 'subject'], ascending=True).to_csv(
        os.path.join(results_dir, 'Day1_means_bysubject_wide.csv'))
    df2_wide.sort_values(['group', 'subject'], ascending=True).to_csv(
        os.path.join(results_dir, 'Day2_means_bysubject_wide.csv'))

    return df_means, df1_means, df2_means


def prepare_excel_export(df_means, df_meansdur, results_dir, totalids=88, max_days=5):
    # Generate all possible subject IDs
    allids = ['cpvib' + str(item).zfill(3) for item in range(1, totalids + 1)]
    varlist = ['Accuracy', 'MT', 'RT', 'pathlength', 'velPeak', 'EndPointError', 'IDE', 'PLR', 'PLR_2']
    exceltitle = os.path.join(results_dir, 'UL KINARM Mastersheet Auto Format.xlsx')
    exceltitle2 = os.path.join(results_dir, 'UL KINARM Mastersheet Long Format.xlsx')

    # Create a mapping from variable names to their order
    variable_order = {var: idx for idx, var in enumerate(varlist)}

    for thisday in range(1, max_days + 1):
        day_label = 'Day' + str(thisday)
        df1_means = df_means.loc[df_means['day'] == day_label]

        # Pivot to wide format for df_means
        df1_wide = df1_means.pivot_table(
            index=["subject", "visit", "studyid", "group", "day"],
            columns=['Condition', 'Affected'],
            values=varlist
        )

        # Flatten MultiIndex columns
        df1_wide.columns = df1_wide.columns.to_flat_index()

        # Sort columns based on the desired variable order
        df1_wide = df1_wide[sorted(df1_wide.columns, key=lambda x: (
            variable_order.get(x[0], len(varlist)), x[1], x[2]))]

        # Reset index to turn MultiIndex into columns
        df1_wide = df1_wide.reset_index(level=["subject", "visit", "group", "day"])

        # Ensure all subject IDs are present
        missing = list(set(allids) - set(df1_wide['subject'].astype(str)))
        if missing:
            # Create a DataFrame for missing IDs with NaN values
            missing_df = pd.DataFrame({'subject': missing})
            df1_wide = pd.concat([df1_wide, missing_df], ignore_index=True, sort=False)

        df1_wide.index.name = 'NMSKL_ID'
        df1_wide = df1_wide.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'})

        # Process and save to Excel
        sheet_name = f'Day{thisday}_Master_Formatted'
        if thisday == 1:
            with pd.ExcelWriter(exceltitle) as writer:
                df1_wide.to_excel(writer, sheet_name=sheet_name)
        else:
            with pd.ExcelWriter(exceltitle, engine="openpyxl", mode='a') as writer:
                df1_wide.to_excel(writer, sheet_name=sheet_name)

    # Write the long format DataFrame to a separate Excel file
    with pd.ExcelWriter(exceltitle2) as writer:
        df_meansdur.to_excel(writer, sheet_name='AllDays_Master_Formatted')


all_df = pd.read_csv('all_processed_df.csv')
varlist = ['Accuracy','MT','RT','pathlength','velPeak']
# Call the function
df_means, df1_means, df2_means = save_data_to_csv(all_df, varlist)
prepare_excel_export(df_means, df_meansdur, results_dir, totalids=88, max_days=5)



In [85]:
def process_and_save_day_data_std(df_std, df_stddur, day_num, exceltitle, all_ids):
    current_day = f'Day{day_num}'
    
    # Process Day-Wise Wide Format Data for Standard Deviation (means by subject)
    df_day_std = df_std[df_std['day'] == current_day]
    df_day_wide_std = pivot_data_to_wide(df_day_std, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected'], VARLIST)

    # Ensure all expected columns are present in the pivot result for means by subject
    all_expected_columns = ['subject', 'visit', 'studyid', 'group', 'day'] + [
        (var, condition, affected)
        for var in VARLIST
        for condition in ['Interception', 'Reaching']
        for affected in ['Less Affected', 'More Affected']
    ]
    
    # Reindex to ensure that all expected columns are present, filling missing columns with NaN
    df_day_wide_std = df_day_wide_std.reindex(columns=all_expected_columns, fill_value=np.nan)
    df_day_wide_std = reindex_with_missing_ids(df_day_wide_std, all_ids)
    df_day_wide_std.rename(columns={'subject': 'KINARM_ID', 'day': 'Visit_day'}, inplace=True)

    # Process Day-Wise Data for Standard Deviation by Duration
    df_day_stddur = df_stddur[df_stddur['day'] == current_day]
    df_day_wide_stddur = pivot_data_to_wide(df_day_stddur, ['subject', 'visit', 'studyid', 'group', 'day'], ['Condition', 'Affected', 'Duration'], VARLIST)

    # Ensure all expected columns for durations are present in the pivot result
    all_expected_columns_dur = ['subject', 'visit', 'studyid', 'group', 'day'] + [
        (var, condition, affected, duration)
        for var in VARLIST
        for condition in ['Interception', 'Reaching']
        for affected in ['Less Affected', 'More Affected']
        for duration in [500, 625, 750, 900]  # Replace with the actual durations you expect
    ]
    
    # Reindex to ensure that all expected columns for duration are present, filling missing columns with NaN
    df_day_wide_stddur = df_day_wide_stddur.reindex(columns=all_expected_columns_dur, fill_value=np.nan)
    df_day_wide_stddur = reindex_with_missing_ids(df_day_wide_stddur, all_ids)
    df_day_wide_stddur.drop(columns=['subject', 'visit', 'day', 'group'], inplace=True)

    # Combine the wide format data for means by subject and duration
    df_day_combo_std = pd.concat([df_day_wide_std, df_day_wide_stddur], axis=1, join="inner")

    # Write the combined data to Excel
    if day_num == 1:
        save_excel(df_day_combo_std, exceltitle, f'{current_day}_Master_Formatted_STD', mode='w')
    else:
        save_excel(df_day_combo_std, exceltitle, f'{current_day}_Master_Formatted_STD', mode='a')
